In [3]:
pip install nltk rouge_score

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge_score'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24987 sha256=47e28e33dee25b88bb6b52032345cf0848cdc64ab809bb61a3f50b3550610ab4
  Stored in directory: /Users/zinmarmyint/Library/Caches/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated package

In [9]:
import json
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import re

def parse_custom_format(file_path):
    """
    ## 1, ## 2 ကဲ့သို့သော ခေါင်းစီးများပါဝင်သည့်
    အထူးဖိုင်ပုံစံကို ဖတ်ပြီး JSON data များ ထုတ်ယူသည်။
    """
    data_dict = {}
    current_key = None
    json_lines = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip().startswith('##'):
                # ခေါင်းစီးအသစ်တွေ့ပြီ။
                if current_key and json_lines:
                    try:
                        json_str = "".join(json_lines)
                        data_dict[current_key] = json.loads(json_str)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON for {current_key}: {e}")
                
                # key အသစ်ကို ယူသည်။
                parts = line.strip().split()
                if len(parts) > 1:
                    key_num = parts[1]
                    current_key = f'image_{key_num}'
                json_lines = []
            elif current_key:
                # လက်ရှိ key အတွက် JSON line တွေကို စုသည်။
                json_lines.append(line)

    # နောက်ဆုံး block ကို ထည့်သွင်းသည်။
    if current_key and json_lines:
        try:
            json_str = "".join(json_lines)
            data_dict[current_key] = json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for {current_key}: {e}")

    return data_dict

def clean_text(text):
    """
    စာသားထဲက မလိုလားအပ်တဲ့ အရာတွေကို ဖယ်ရှားပေးပါတယ်။
    """
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[\u4e00-\u9fff\u3040-\u30ff]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

def evaluate_model(candidate_data, reference_data):
    """
    မော်ဒယ်ရဲ့ output နဲ့ reference data ကို နှိုင်းယှဉ်ပြီး အကဲဖြတ်ပေးပါတယ်။
    """
    caption_bleu_scores = []
    caption_rouge_l_scores = []
    description_bleu_scores = []
    description_rouge_l_scores = []

    smoothie = SmoothingFunction().method1
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    for i in range(1, 13):
        image_key = f'image_{i}'
        
        if image_key not in candidate_data:
            print(f"Warning: Data for {image_key} not found. Skipping...")
            continue

        candidate_caption = candidate_data[image_key]['caption']['my']
        reference_caption = reference_data[image_key]['caption']['my']

        candidate_description = candidate_data[image_key]['description']['my']
        reference_description = reference_data[image_key]['description']['my']

        # BLEU score အတွက် စကားလုံးအလိုက် ခွဲထုတ်သည်။
        candidate_caption_tokens = candidate_caption.split()
        reference_caption_tokens = [reference_caption.split()]
        candidate_description_tokens = candidate_description.split()
        reference_description_tokens = [reference_description.split()]

        caption_bleu = sentence_bleu(reference_caption_tokens, candidate_caption_tokens, smoothing_function=smoothie)
        description_bleu = sentence_bleu(reference_description_tokens, candidate_description_tokens, smoothing_function=smoothie)

        # ROUGE score အတွက် စာသားများကို clean လုပ်သည်။
        cleaned_candidate_caption = clean_text(candidate_caption)
        cleaned_reference_caption = clean_text(reference_caption)
        cleaned_candidate_description = clean_text(candidate_description)
        cleaned_reference_description = clean_text(reference_description)
        
        caption_rouge = scorer.score(cleaned_reference_caption, cleaned_candidate_caption)['rougeL']
        description_rouge = scorer.score(cleaned_reference_description, cleaned_candidate_description)['rougeL']
        
        caption_rouge_l_scores.append(caption_rouge.fmeasure)
        description_rouge_l_scores.append(description_rouge.fmeasure)
        caption_bleu_scores.append(caption_bleu)
        description_bleu_scores.append(description_bleu)

    avg_caption_bleu = sum(caption_bleu_scores) / len(caption_bleu_scores) if caption_bleu_scores else 0
    avg_caption_rouge_l = sum(caption_rouge_l_scores) / len(caption_rouge_l_scores) if caption_rouge_l_scores else 0
    avg_description_bleu = sum(description_bleu_scores) / len(description_bleu_scores) if description_bleu_scores else 0
    avg_description_rouge_l = sum(description_rouge_l_scores) / len(description_rouge_l_scores) if description_rouge_l_scores else 0

    return {
        'Caption_BLEU': avg_caption_bleu,
        'Caption_ROUGE-L': avg_caption_rouge_l,
        'Description_BLEU': avg_description_bleu,
        'Description_ROUGE-L': avg_description_rouge_l
    }

if __name__ == '__main__':
    # သင့်ရဲ့ reference data ကို ဒီမှာ ထည့်ပါ။
    # အကယ်၍ အပြင်ဖိုင်မှာ ထားပါက json.load() နဲ့ ဖတ်ယူနိုင်ပါသည်။
    reference_data = {
        "image_1": {
            "caption": {"my": "အရိပ်က ဖျောက်ထားတဲ့ အန္တရာယ်ကို ဖော်ထုတ်တယ်။"},
            "description": {"my": "လူတစ်ဦးက ကားရှေ့ကိုဖြတ်ကူးမယ်လို့ စိတ်ထားနေတယ်။ ကားရဲ့ အရိပ်က သေနတ်ပုံစံထွက်နေပြီး ဖျောက်ထားတဲ့ အန္တရာယ် သို့မဟုတ် ခြိမ်းခြောက်မှုကို အဓိပ္ပာယ်ဖော်ပြနေတယ်။"}
        },
        "image_2": {
            "caption": {"my": "သင့်ရဲ့ လုံခြုံရေးနဲ့ ဖြေရှင်းချက်နှစ်ခုစလုံးမှာ အပေါက်တွေပါလာတဲ့အခါ။"},
            "description": {"my": "ကာတွန်းကို အကွက်နှစ်ခုခွဲထားပါတယ်။ အပေါ်ပုံမှာ လူတစ်ယောက်က ယိုနေတဲ့ထီးကို ကိုင်ထားပြီး ငှက်အိမ်ထဲက ငှက်တစ်ကောင်က သူ့ကို နောက်ထီးတစ်ချောင်းရောင်းဖို့ ကြိုးစားနေပါတယ်။ အောက်ပုံမှာတော့ ငှက်ရဲ့ထီးရော၊ လူရဲ့ထီးအသစ်ရော နှစ်ခုစလုံး ပျက်စီးနေပြီး မိုးတွေက နှစ်ခုစလုံးပေါ်ကို သည်းသည်းမည်းမည်း ရွာနေပါတယ်။"}
        },
        "image_3": {
            "caption": {"my": "အချို့ထီးတွေက ပိုကျယ်ပြန့်တဲ့ ကာကွယ်မှုကို ပေးတယ်။"},
            "description": {"my": "မိုးသည်းထန်စွာရွာသွန်းနေချိန်တွင် လူများသည် သာမန်ထီးများကို ကိုင်ဆောင်၍ လမ်းလျှောက်နေကြပါသည်။ သို့သော် လူတစ်ဦးမှာ နံပါတ် ၄၉၃ ဟုရေးထားသည့် တဲပုံစံဧရာမထီးကြီးကို ထိန်းချုပ်ရန် ရုန်းကန်နေရပါသည်။"}
        },
        "image_4": {
            "caption": {"my": "လေယာဉ်နဲ့ ခရီးသွားခြင်းက တစ်ချိန်က အေးချမ်းတဲ့လွတ်မြောက်မှုတစ်ခုဖြစ်ခဲ့တယ်။"},
            "description": {"my": "လေယာဉ်ပေါ်တွင် ခရီးသည်များစွာ ထိုင်နေကြပြီး လူတိုင်း ဖုန်းပြောခြင်းတွင် နစ်မြုပ်နေကြပါသည်။ ညာဘက်အစွန်ဆုံးရှိ အမျိုးသားသည် စိတ်ပျက်အားငယ်နေပုံရပြီး ဆူညံသံများကို လက်ဖြင့်ပိတ်ကာ ကာကွယ်ရန် ကြိုးစားနေသည်။"}
        },
        "image_5": {
            "caption": {"my": "အခုကျွန်တော့်အသံကို ကြားရလား။ ဒါက စမ်းသပ်မှုတစ်ခုပါ။"},
            "description": {"my": "အမျိုးသမီးတစ်ဦးက လက်ပိုက်ပြီး ကုလားထိုင်မှာ ထိုင်နေတဲ့အမျိုးသားကို စူးစိုက်ကြည့်နေပါတယ်။ မျက်မှန်တပ်ထားတဲ့အမျိုးသားက သတင်းစာတစ်စောင်ကို ကိုင်ထားပြီး သူမကို လျစ်လျူရှုထားပုံရပါတယ်။"}
        },
        "image_6": {
            "caption": {"my": "လက်စားချေတာက ဒေါသထွက်နေတဲ့လူအုပ်နဲ့အတူတူဆို ပိုအရသာရှိတယ်။"},
            "description": {"my": "ကာတွန်းဇာတ်လမ်းတိုလေးတစ်ခုတွင် ယောက်ျားလေးတစ်ဦးက နောက်တစ်ဦးကို တစ်ခုခုနဲ့ ပစ်လိုက်ရာ မျက်နှာကို ထိမှန်သွားသည်။ ထို့နောက် သားကောင်ဖြစ်သူက ပထမကောင်လေးကို ဒေါသထွက်နေသောလူအုပ်နှင့်အတူ လိုက်ဖမ်းရန် စိတ်ကူးယဉ်၍ လက်စားချေရန် ကြံစည်သည်။ သူသည် ယခုအခါ ဒေါသထွက်နေသောလူအုပ်မှ ထွက်ပြေးနေရသည်။"}
        },
        "image_7": {
            "caption": {"my": "မုန်းတီးမှုဆိုတာ မချနိုင်တဲ့ လက်နက်တစ်ခုပါ။"},
            "description": {"my": "လူနှစ်ယောက် ပြင်းပြင်းထန်ထန် ငြင်းခုန်နေကြပြီး သူတို့ရဲ့ ခေါင်းတွေက ပုံပျက်ပန်းပျက် ဖောင်းကားလာကာ ကိုယ်ခန္ဓာကနေ ပြုတ်ထွက်သွားပါတယ်။ ခေါင်းတွေမရှိတော့မှ သူတို့ဟာ အေးအေးဆေးဆေး လက်ဆွဲနှုတ်ဆက်နိုင်ခဲ့ကြပြီး ငြိမ်းချမ်းရေးငှက်တစ်ကောင်ဟာ သူတို့ကြားထဲမှာ ပျံသန်းသွားပါတယ်။"}
        },
        "image_8": {
            "caption": {"my": "တချို့ဦးထုပ်တွေက တခြားဟာတွေထက် မိုးရေကို ပိုဖမ်းနိုင်တယ်။"},
            "description": {"my": "မိုးသည်းထန်စွာရွာသွန်းနေချိန်တွင် အမျိုးသားနှစ်ဦးသည် မိမိကိုယ်ကို အကာအကွယ်ယူရန် ကြိုးစားနေကြသည်။ အမျိုးသားတစ်ဦးက ပြုံးနေပြီး သူ၏ဦးထုပ်အောက်တွင် ခြောက်သွေ့နေပုံရသော်လည်း၊ နောက်တစ်ဦး၏ဦးထုပ်ထဲတွင် ရေများပြည့်နေပြီး မျက်နှာပေါ်သို့ အစက်အစက်ကျနေသည်။"}
        },
        "image_9": {
            "caption": {"my": "ဥယျာဉ်ကိုရေလောင်းတာ...ရေပိုက်ပါ အရည်ပျော်သွားတယ်။"},
            "description": {"my": "လယ်သမားတစ်ဦးသည် အလွန်ပူပြင်းသောနေရောင်အောက်တွင် ဥယျာဉ်ကို ရေလောင်းရန် ကြိုးစားနေသည်။ အပူဒဏ်ပြင်းထန်လွန်းသဖြင့် ပလတ်စတစ်ရေပိုက်သည် အရည်ပျော်ပြီး သူ၏လက်နှင့် ကပ်သွားခဲ့သည်။"}
        },
        "image_10": {
            "caption": {"my": "သင့်ရဲ့ လုံခြုံရေးနဲ့ ဖြေရှင်းချက်နှစ်ခုစလုံးမှာ အပေါက်တွေပါလာတဲ့အခါ။"},
            "description": {"my": "ကာတွန်းကို အကွက်နှစ်ခုခွဲထားပါတယ်။ အပေါ်ပုံမှာ လူတစ်ယောက်က ယိုနေတဲ့ထီးကို ကိုင်ထားပြီး ငှက်အိမ်ထဲက ငှက်တစ်ကောင်က သူ့ကို နောက်ထီးတစ်ချောင်းရောင်းဖို့ ကြိုးစားနေပါတယ်။ အောက်ပုံမှာတော့ ငှက်ရဲ့ထီးရော၊ လူရဲ့ထီးအသစ်ရော နှစ်ခုစလုံး ပျက်စီးနေပြီး မိုးတွေက နှစ်ခုစလုံးပေါ်ကို သည်းသည်းမည်းမည်း ရွာနေပါတယ်။"}
        },
        "image_11": {
            "caption": {"my": "တချို့ကျောင်းသားတွေက သူတို့အတွက် အကျိုးမရှိလောက်အောင်ကို တော်ကြတယ်။"},
            "description": {"my": "ဆရာမတစ်ဦးသည် 'ဆီးနှင်းဖြူနှင့် လူပုလေးခုနစ်ယောက်' စာအုပ်ကို ကလေးအုပ်စုတစ်စုအား ဖတ်ပြနေပါသည်။ ကောင်လေးတစ်ယောက်ကတော့ သူ့ရဲ့လက်ချောင်းခုနစ်ချောင်းကိုထောင်ပြီး ကျောင်းသားခြောက်ယောက်ပဲ ရှိကြောင်း ဆရာမကို ပြင်ပေးနေပါတယ်။"}
        },
        "image_12": {
            "caption": {"my": "ဖားခြေထောက်က ဒီမှာ အရမ်းနာမည်ကြီးတဲ့ ဟင်းတစ်မျိုးပါ။"},
            "description": {"my": "စားသောက်ဆိုင်တစ်ဆိုင်တွင် စုံတွဲတစ်တွဲသည် မီနူးများကို ကြည့်နေကြသည်။ အမျိုးသမီး၏နောက်တွင် 'TRY OUR FROGS' LEGS' (ကျွန်ုပ်တို့၏ ဖားခြေထောက်များကို မြည်းစမ်းကြည့်ပါ) ဟုရေးထားသော ဆိုင်းဘုတ်တစ်ခုရှိသည်။ ရှေ့မြင်ကွင်းတွင် ဘီးသေးသေးလေးပါသော ဖားတစ်ကောင်သည် နောက်ဖေးတံခါးမှ လှိမ့်ထွက်သွားသည်။"}
        }
    }
    
    file_paths = {
        'chatGPT': 'chatGPT.txt',
        'Claude': 'Claude.txt',
        'Gemini': 'Gemini.txt'
    }

    results = {}
    for model_name, file_path in file_paths.items():
        print(f"Evaluating {model_name}...")
        candidate_data = parse_custom_format(file_path)
        if candidate_data:
            model_results = evaluate_model(candidate_data, reference_data)
            if model_results:
                results[model_name] = model_results
    
    if results:
        results_df = pd.DataFrame(results).T
        print("\nEvaluation Results:")
        print(results_df)
    else:
        print("\nNo models were successfully evaluated. Please check your file formats or file paths.")

Evaluating chatGPT...
Evaluating Claude...
Evaluating Gemini...

Evaluation Results:
         Caption_BLEU  Caption_ROUGE-L  Description_BLEU  Description_ROUGE-L
chatGPT      0.083333              0.0          0.085233             0.000000
Claude       0.000000              0.0          0.008076             0.083333
Gemini       0.786743              0.0          0.869292             0.083333
